# KorBertSum 문서 추출요약

## 1. Preparing Datasets

### (1) Pre-trained BERT Model

BERT fine-tuning을 위해서는 먼저 사전학습된 BERT 모델이 필요

- [ETRI 홈페이지](https://aiopen.etri.re.kr/service_dataset.php)에서 모델 사용신청을 하여 BERT Model을 다운로드

```
├─ 001_bert_morp_pytorch
│   ├── KorBERT_FAQ_20190619.pdf
│   ├── bert_config.json
│   ├── pytorch_model.bin
│   ├── readme.txt
│   ├── src_examples
│   ├── src_tokenizer
│   └── vocab.korean_morp.list
└── 002_bert_morp_tensorflow
    ├── KorBERT_FAQ_20190619.pdf
    ├── bert_config.json
    ├── model.ckpt.data-00000-of-00001
    ├── model.ckpt.index
    ├── model.ckpt.meta
    ├── src_tokenizer
    └── vocab.korean_morp.list

```

### (2) Train Dataset

1. [Dacon 문서 추출요약 AI 경진대회](https://dacon.io/competitions/official/235671/data) 의 데이터 셋 활용 _-> 현재 대회종료로 data 제공 안하고 있음_

    **train.jsonl** :  학습에 사용 할 데이터셋

    - media : 기사 미디어

    - id : 각 데이터 고유 번호

    - article_original : 전체 기사 내용, 문장별로 split되어 있음

    - abstractive : 사람이 생성한 요약문

    - extractive : 사람이 추출한 요약문 3개의 index

2. [AI Hub](https://aihub.or.kr/aidata/8054)의 데이터 셋 활용
- 해당 json 데이터를 대회에서 쓰인 데이터 형태로 파싱해서 사용

```
├── Training
│   ├── train_original.json
│   ├── 법률_train_original.zip
│   ├── 사설_train_original.zip
│   └── 신문기사_train_original.zip
└── Validation
    ├── valid_original.json
    ├── 법률_valid_original.zip
    ├── 사설_valid_original.zip
    └── 신문기사_valid_original.zip
```

## 2. Preprocess 

### (1) sample.jsonl 파일 확인

In [11]:
cd /Users/imok/workspace/github/imOk/AI/KorBertSum

/Users/imok/workspace/github/imOk/AI/KorBertSum


In [12]:
import json

with open('../sample.jsonl', 'r') as json_file:
    json_list = list(json_file)
result = []
for json_str in json_list:
    result.append(json.loads(json_str))

In [13]:
import pandas as pd
df = pd.DataFrame(result)
df.head()

,id,abstractive,extractive,article_original,media
0,353465974,충주시는 민간보조사업의 증가와 보조금 집행관리에 대한 부당 행위가 증가함에따라 15...,"[2, 3, 5]","[보조금 집행 위법행위·지적사례 늘어, 특별감사반, 2017~2018년 축제 점검,...",충청투데이
1,366398381,국무조정실은 8일 오후 대전시청에서 '대전지역 규제혁신 현장간담회'를 열고 대전과 ...,"[4, 6, 14]","[8일 대전시청에서 규제혁신 간담회, 도시개발 산업용지도 특화단지 지정가능, 국무조...",중도일보
2,360025161,중국 경제일간지 21세기경제보도는 중국 대형 생명보험사인 차이나라이프가 '차이나라이...,"[0, 1, 2]",[중국 경제일간지 21세기경제보도는 중국 대형 생명보험사인 차이나라이프(中國人壽)가...,내일신문
3,361884128,1일 대검찰청은 '조속한 검찰개혁 방안을 마련하라'는 문재인 대통령의 지시에 따라...,"[4, 5, 3]","[전승표 기자, 대검, 文 지시에 발빠른 방안 마련 서울중앙지검 3곳 빼고 모두 폐...",기호일보
4,351452460,제주도가 민선 7기 출범과 함께 조직개편을 추진하면서 지난해 8월 공무원 정원을 2...,"[6, 11, 7]",[제주도가 공무원 정원 102명 증원을 추진하고 있는 가운데 제주도청 조직 및 인력...,한라일보


In [14]:
df.article_original[0]

['보조금 집행 위법행위·지적사례 늘어',
 '특별감사반, 2017~2018년 축제 점검',
 '충주시가 민간에게 지원되는 보조사업의 대형축제와 관련해 선정·집행·정산 등 운영실태 전반에 대한 자체 감사를 실시할 계획이라고 밝혔다.',
 '이는 최근 민간보조사업의 증가와 더불어 보조금 집행관리에 대한 위법 부당 행위와 지적사례가 지속적으로 증가함에 따라, 감사를 통해 취약요인을 점검해 올바른 보조금 사용 풍토를 정착시키겠다는 취지다.',
 '시는 감사담당관실과 기획예산과 보조금 관련 주무관으로 특별감사반을 편성해 2017년부터 2018년까지 집행된 축제성 보조금 집행에 대한 철저한 점검과 감사를 통해 부정 수급 및 부정 집행이 확인되면 엄정한 조치를 취할 방침이다.',
 '시는 지난 15일부터 25일까지 10일간의 사전감사를 통해 보조금 실태를 파악한 후, 8월15일까지 세부감사를 진행할 예정이라고 전했다.',
 '축제성 관련 부정수급 유형을 보면 허위·기타 부정한 방법으로 보조금 신청, 사업 실적을 부풀려 보조금을 횡령·편취, 보조금 교부 목적과 다른 용도로 집행, 보조금으로 취득한 재산에 대해 지자체장의 승인없이 임의 처분 등이 해당된다.',
 "시는 불법보조금 근절과 효율적인 점검 및 적극적인 시민관심을 유도하기 위해 '지방보조금 부정수급 신고센터(☏850-5031)'를 설치 운영하고 있다.",
 '지방보조금 부정수급 신고 시 직접방문 및 국민신문고(www.epeople.or.kr), 충주시홈페이지(www.chungju.or.kr)를 통해 접수하면 되고, 신고취지와 이유를 기재하고 부정행위와 관련한 증거자료를 제시하면 된다.',
 '단, 익명 신고는 접수치 않는다.',
 '시 관계자는 "이번 자체 점검 및 감사를 통해 축제보조금이 제대로 쓰이는지에 대한 반성과 함께 보조금 집행의 투명성 및 행정의 신뢰성을 확보하는데 최선을 다하겠다"고 말했다.',
 '한편. 시는 감사 및 예산부서 합동으로 컨설팅 위주의 상반기 보조금 특정감사(1월10일~20일)를 실시해

---

### (2) 신문기사 train data 전처리

In [15]:
import json
from collections import OrderedDict 
import pprint

with open('../text/Training/train_original.json') as json_file:
    jsonObject = json.load(json_file)

In [16]:
jsonArray = jsonObject['documents']
jsonArray[:1]

[{'id': '290741778',
  'category': '종합',
  'media_type': 'online',
  'media_sub_type': '지역지',
  'media_name': '광양신문',
  'size': 'small',
  'char_count': '927',
  'publish_date': '2018-01-05 18:54:55',
  'title': '논 타작물 재배, 2월 말까지 신청하세요',
  'text': [[{'index': 0,
     'sentence': 'ha당 조사료 400만원…작물별 차등 지원',
     'highlight_indices': ''}],
   [{'index': 1,
     'sentence': '이성훈 sinawi@hanmail.net',
     'highlight_indices': ''}],
   [{'index': 2,
     'sentence': '전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다.',
     'highlight_indices': ''}],
   [{'index': 3,
     'sentence': '쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다.',
     'highlight_indices': '35,37'}],
   [{'index': 4,
     'sentence': '올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다.',
     'highlight_indices': '9,11;33,34'},
    {'index': 5,
     'sentence': '지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다.',
     'highlight_indices': '50,52'}],
   [{

In [7]:
id = []
for name in jsonArray:
    id.append(name['id'])
id[:5]

['290741778', '290741792', '290741793', '290741794', '290741797']

In [8]:
media = []
for name in jsonArray:
    media.append(name['media_name'])
media[:5]

['광양신문', '광양신문', '광양신문', '광양신문', '광양신문']

In [9]:
abstractive = []
for name in jsonArray:
    abstractive.append(*name['abstractive'])
abstractive[:5]

["전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 '쌀 생산조정제'를 적극적으로 시행하기로 하고 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에서 신청받는다 .",
 '여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사벽화마을과 자산마을은 알록달록 색깔 옷을 입었고 사업 시행과 준공 과정에서도 주민들의 참여를 유도해 경관사업의 좋은 사례를 만들었다.',
 '전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단 목표 달성을 위한 새해 각오를 다졌다.',
 '광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작목 중심으로 농업인 실용교육을 실시한다.',
 '올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항해 전남의 관광지를 방문할 예정이다.']

In [10]:
extractive = []
for name in jsonArray:
    extractive.append(name['extractive'])
extractive[:5]

[[2, 3, 10], [2, 4, 11], [3, 5, 7], [2, 3, 4], [3, 7, 4]]

In [11]:
article_original = []
for name in jsonArray:
    article_original.append(name['text'])

In [12]:
article_original[0][0][0]['sentence']

'ha당 조사료 400만원…작물별 차등 지원'

In [102]:
total_article = []
for i, article in enumerate(article_original):
    article_sent = []
    for j, art in enumerate(article):
        article_text = []
        for k, text in enumerate(art):
            article_text.append(text['sentence'])
        a_t = ','.join(article_text)
        article_sent.append(a_t)
    total_article.append(article_sent)

In [103]:
total_article[0]

['ha당 조사료 400만원…작물별 차등 지원',
 '이성훈 sinawi@hanmail.net',
 '전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다.',
 '쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다.',
 '올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다.,지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다.',
 '지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다.',
 '농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다.',
 'ha(3000평)당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 등 두류작물은 280만원 등이다.,벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다.',
 '논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다.',
 '전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다.',
 '최향철 전라남도 친환경농업과장은 “최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다.']

In [104]:
df = pd.DataFrame()

In [105]:
df['media'] = media
df['id'] = id
df['article_original'] = total_article
df['article_morp'] = 0
df['abstractive'] = abstractive
df['extractive'] = extractive

In [106]:
df.head(2)

,media,id,article_original,article_morp,abstractive,extractive
0,광양신문,290741778,"[ha당 조사료 400만원…작물별 차등 지원, 이성훈 sinawi@hanmail.n...",0,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...,"[2, 3, 10]"
1,광양신문,290741792,"[8억 투입, 고소천사벽화·자산마을에 색채 입혀, 이성훈 sinawi@hanmail...",0,여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...,"[2, 4, 11]"


- csv 파일 예시 <br>

| media  | id       | article_original | article_morp      | abstractive | extractive |
|:------:|:--------:|:----------------:|:-----------------:|:-----------:|:----------:|
| 신문사 | 기사번호 | 기사원문         | 형태소분석된 기사 | 생성요약    | 추출요약   |

In [107]:
df.shape

(243983, 6)

In [108]:
df.isnull().sum()

media               0
id                  0
article_original    0
article_morp        0
abstractive         0
extractive          0
dtype: int64

In [110]:
import re
def clean_text(text): 
    text = re.sub(r'(\()(.*?)(\))', '', str(text))  # 소괄호 (세부 설명)
    text = re.sub(r'[가-힣]+ ([\w\.\_\-])*[a-zA-Z0-9]+([\w\.\_\-])*([a-zA-Z0-9])+([\w\.\_\-])+@([a-zA-Z0-9]+\.)+[a-zA-Z0-9]{2,8}','',text) # 기자 이메일 제거
    text = re.sub(r'\'\'','',text) # 공백 제거
    text = re.sub(r'[?!]', '.', text)          # ?! => 마침표 처리
    text = re.sub(r'[\·\:\-\_\…]', ' ', text)  # 문장부호 구분자 => 공백 처리
    text = re.sub(r'\s+', ' ', text) #remove extra space
    text = re.sub(r'^\s+', '', text) #remove space from start
    text = re.sub(r'\s+$', '', text) #remove space from the end
    text = re.sub('\s{2,}', ' ', text)        # 2번 이상의 space 제거
    text = text.strip()
    return text

In [122]:
from tqdm import tqdm

for i in tqdm(range(len(df))):
    arr = []
    for j in df['article_original'][i]:
        arr.append(clean_text(j))
    arr = [x for x in arr if x]
    df['article_original'][i] = arr

  0%|                                                                        | 0/243983 [00:00<?, ?it/s]/var/folders/m6/hrff0b7x4hl1m5j8nxchlk5c0000gn/T/ipykernel_4436/2284716926.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['article_original'][i] = arr
100%|██████████████████████████████████████████████████████████| 243983/243983 [09:06<00:00, 446.04it/s]


In [123]:
df['article_original'][0]

['ha당 조사료 400만원 작물별 차등 지원',
 '전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다.',
 '쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다.',
 '올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다.,지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a 이상 벼 이외 다른 작물을 재배한 농업인이다.',
 '지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다.',
 '농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다.',
 'ha당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩 팥 등 두류작물은 280만원 등이다.,벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다.',
 '논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다.',
 '전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다.',
 '최향철 전라남도 친환경농업과장은 “최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다.']

In [124]:
from konlpy.tag import Mecab

mecab = Mecab()
# 형태소 분석
mecab.morphs('전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다.')

['전라남도',
 '가',
 '쌀',
 '과잉',
 '문제',
 '를',
 '근본',
 '적',
 '으로',
 '해결',
 '하',
 '기',
 '위해',
 '올해',
 '부터',
 '시행',
 '하',
 '는',
 '쌀',
 '생산',
 '조정',
 '제',
 '를',
 '적극',
 '추진',
 '키로',
 '했',
 '다',
 '.']

In [126]:
for i in tqdm(range(len(df))):
    arr = []
    for j in df['article_original'][i]:
        arr.append(' '.join(mecab.morphs(j)))
    df['article_morp'][i] = arr

  0%|                                                                        | 0/243983 [00:00<?, ?it/s]/var/folders/m6/hrff0b7x4hl1m5j8nxchlk5c0000gn/T/ipykernel_4436/4120418180.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['article_morp'][i] = arr
/opt/homebrew/Caskroom/miniconda/base/envs/imok/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
100%|█████████████████████████████████████████████████████████| 243983/243983 [03:21<00:00, 1210.22it/s]


In [127]:
df.article_morp[0]

['ha 당 조사료 400 만 원 작물 별 차등 지원',
 '전라남도 가 쌀 과잉 문제 를 근본 적 으로 해결 하 기 위해 올해 부터 시행 하 는 쌀 생산 조정 제 를 적극 추진 키로 했 다 .',
 '쌀 생산 조정 제 는 벼 를 심 었 던 논 에 벼 대신 사료 작물 이나 콩 등 다른 작물 을 심 으면 벼 와 의 일정 소득 차 를 보전 해 주 는 제도 다 .',
 '올해 전남 의 논 다른 작물 재배 계획 면적 은 전국 5 만 ha 의 약 21 % 인 1 만 698 ha 로 , 세부 시행 지침 을 확정 , 시군 에 통보 했 다 . , 지원 사업 대상 은 2017 년 산 쌀 변동 직 불금 을 받 은 농지 에 10 a 이상 벼 이외 다른 작물 을 재배 한 농업 인 이 다 .',
 '지원 대상 작물 은 1 년 생 을 포함 한 다년생 의 모든 작물 이 해당 되 나 재배 면적 확대 시 수급 과잉 이 우려 되 는 고추 , 무 , 배추 , 인삼 , 대파 등 수급 불안 품목 은 제외 된다 .',
 '농지 의 경우 도 이미 다른 작물 재배 의무 가 부여 된 간척지 , 정부 매입 비축 농지 , 농진청 시범 사업 , 경관 보전 직 불금 수령 농지 등 은 제외 될 예정 이 다 .',
 'ha 당 지원 단가 는 평균 340 만 원 으로 사료 작물 400 만 원 , 일반 작물 은 340 만 원 , 콩 팥 등 두류 작물 은 280 만 원 등 이 다 . , 벼 와 소득 차 와 영농 편이 성 을 감안 해 작물 별 로 차등 지원 된다 .',
 '논 에 다른 작물 재배 를 바라 는 농가 는 오 는 22 일 부터 2 월 28 일 까지 농지 소재지 읍 면 동사무소 에 신청 해야 한다 .',
 '전남 도 는 도와 시군 에 관련 기관 과 농가 등 이 참여 하 는 ‘ 논 타 작물 지원 사업 추진 협의회 ’ 를 구성 , 지역 특성 에 맞 는 작목 선정 및 사업 심의 등 을 본격 추진 할 방침 이 다 .',
 '최향 철 전라남도 친환경 농업 과장 은 “ 최근 쌀값 이 다소 상승 추세 에 있 으나 매년 공급 

In [128]:
df.abstractive[0]

"전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 '쌀 생산조정제'를 적극적으로 시행하기로 하고 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에서 신청받는다 ."

In [129]:
df.article_original[0]

['ha당 조사료 400만원 작물별 차등 지원',
 '전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다.',
 '쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다.',
 '올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다.,지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a 이상 벼 이외 다른 작물을 재배한 농업인이다.',
 '지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다.',
 '농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다.',
 'ha당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩 팥 등 두류작물은 280만원 등이다.,벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다.',
 '논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다.',
 '전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다.',
 '최향철 전라남도 친환경농업과장은 “최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다.']

In [130]:
# csv로 저장
df.to_csv('/Users/imok/workspace/github/imOk/AI/train_result.csv')

### (3) 신문 train 데이터 csv to json

In [17]:
test = pd.read_csv('/Users/imok/workspace/github/imOk/AI/train_result.csv').drop('Unnamed: 0', axis=1)
test.head()

,media,id,article_original,article_morp,abstractive,extractive
0,광양신문,290741778,"['ha당 조사료 400만원 작물별 차등 지원', '전라남도가 쌀 과잉문제를 근본적...","['ha 당 조사료 400 만 원 작물 별 차등 지원', '전라남도 가 쌀 과잉 문...",전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...,"[2, 3, 10]"
1,광양신문,290741792,"['8억 투입, 고소천사벽화 자산마을에 색채 입혀', '여수시는 원도심 일대에서 추...","['8 억 투 입 , 고소 천사 벽화 자산 마을 에 색채 입혀', '여수시 는 원 ...",여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...,"[2, 4, 11]"
2,광양신문,290741793,"['전남드래곤즈 해맞이 다짐 선수 영입 활발', '전남드래곤즈는 지난 4일 구봉산 ...","['전남 드래곤즈 해맞이 다짐 선수 영입 활발', '전남 드래곤즈 는 지난 4 일 ...",전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...,"[3, 5, 7]"
3,광양신문,290741794,"['11~24일, 매실 감 참다래 등 지역특화작목', '광양시는 오는 11일부터 2...","['11 ~ 24 일 , 매실 감 참 다래 등 지역 특화 작목', '광양시 는 오 ...","광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작...","[2, 3, 4]"
4,광양신문,290741797,"['홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항', '타이완의 크루즈관광객...","['홍콩 크루즈 선사 ‘ 아쿠아 리우스 ’ 4 , 6 월 여수항 입항', '타이완 ...",올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...,"[3, 7, 4]"


In [18]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(test, test_size = 0.2)
valid_set, test_set = train_test_split(test_set, test_size = 0.1)

In [19]:
import ast
from tqdm import tqdm
import os

list_dic = []
for idx, row in train_set.iterrows():
    raw = row['article_morp']
    target_idx = ast.literal_eval(row['extractive'])

    sentences = raw.split(',')
    src = [i.split(' ') for i in sentences]
    tgt = [a for i,a in enumerate(src) if i in target_idx]
  
    mydict = {}
    mydict['src'] = src
    mydict['tgt'] = tgt
    list_dic.append(mydict)
        
temp = []
for i,a in enumerate(tqdm(list_dic)):
    if (i+1)%6!=0:
        temp.append(a)
    else:
        filename = 'korean.'+'train'+'.'+str(i//6)+'.json'
        with open('/Users/imok/workspace/github/imOk/AI/KorBertSum'+'/json_data/'+filename, "w", encoding='utf-8') as json_file:
            json.dump(temp, json_file, ensure_ascii=False)
        temp = []

100%|███████████████████████████████████████████████| 195186/195186 [00:37<00:00, 5219.11it/s]


In [20]:
try:
    os.mkdir('/Users/imok/workspace/github/imOk/AI/KorBertSum/json_data/val')
except:
    pass

list_dic = []
for idx, row in valid_set.iterrows():
    raw = row['article_morp']
    target_idx = ast.literal_eval(row['extractive'])

    sentences = raw.split(',')
    src = [i.split(' ') for i in sentences]
    tgt = [a for i,a in enumerate(src) if i in target_idx]

    mydict = {}
    mydict['src'] = src
    mydict['tgt'] = tgt
    list_dic.append(mydict)
        
temp = []
for i,a in enumerate(tqdm(list_dic)):
    
    if (i+1)%6!=0:
        temp.append(a)
    else:
        filename = 'korean.'+'valid'+'.'+str(i//6)+'.json'
        with open('/Users/imok/workspace/github/imOk/AI/KorBertSum'+'/json_data/val/'+filename, "w", encoding='utf-8') as json_file:
            json.dump(temp, json_file, ensure_ascii=False)
        temp = []

100%|█████████████████████████████████████████████████| 43917/43917 [00:07<00:00, 5637.73it/s]


In [21]:
try:
    os.mkdir('/Users/imok/workspace/github/imOk/AI/KorBertSum/json_data/test')
except:
    pass

list_dic = []
for idx, row in valid_set.iterrows():
    raw = row['article_morp']
    target_idx = ast.literal_eval(row['extractive'])

    sentences = raw.split(',')
    src = [i.split(' ') for i in sentences]
    tgt = [a for i,a in enumerate(src) if i in target_idx]

    mydict = {}
    mydict['src'] = src
    mydict['tgt'] = tgt
    list_dic.append(mydict)
        
temp = []
for i,a in enumerate(tqdm(list_dic)):
    
    if (i+1)%6!=0:
        temp.append(a)
    else:
        filename = 'korean.'+'test'+'.'+str(i//6)+'.json'
        with open('/Users/imok/workspace/github/imOk/AI/KorBertSum'+'/json_data/test/'+filename, "w", encoding='utf-8') as json_file:
            json.dump(temp, json_file, ensure_ascii=False)
        temp = []

100%|█████████████████████████████████████████████████| 43917/43917 [00:07<00:00, 5679.96it/s]


### (4) 신문 train 데이터 json to pt

In [24]:
!pwd

/Users/imok/workspace/github/imOk/AI/KorBertSum/src


In [25]:
cd /Users/imok/workspace/github/imOk/AI/KorBertSum/src

/Users/imok/workspace/github/imOk/AI/KorBertSum/src


In [ ]:
%time
!python preprocess.py \
-mode format_to_bert \
-raw_path ../json_data \
-save_path ../bert_data \
-dataset train \
-vocab_file_path ../../model/001_bert_morp_pytorch/vocab.korean_morp.list

In [ ]:
try:
    os.mkdir('/Users/imok/workspace/github/imOk/AI/KorBertSum/bert_data/val')
except:
    pass

!python preprocess.py \
-mode format_to_bert \
-raw_path ../json_data/val \
-save_path ../bert_data \
-dataset valid \
-vocab_file_path ../../model/001_bert_morp_pytorch/vocab.korean_morp.list

In [ ]:
try:
    os.mkdir('/Users/imok/workspace/github/imOk/AI/KorBertSum/bert_data/test')
except:
    pass

!python preprocess.py \
-mode format_to_bert \
-raw_path ../json_data/test \
-save_path ../bert_data \
-dataset test \
-vocab_file_path ../../model/001_bert_morp_pytorch/vocab.korean_morp.list

---

## 3. Train

In [29]:
cd /Users/imok/workspace/github/imOk/AI

/Users/imok/workspace/github/imOk/AI


### (1) package 설치 및 colab GPU 설정

In [3]:
# 기타 패키지 설치
!pip install pytorch_pretrained_bert
!pip install tensorboardX
!pip install transformers
!pip install easydict

     |████████████████████████████████| 123 kB 8.9 MB/s 
     |████████████████████████████████| 132 kB 53.0 MB/s 
     |████████████████████████████████| 8.7 MB 54.0 MB/s 
     |████████████████████████████████| 79 kB 9.8 MB/s 
     |████████████████████████████████| 138 kB 29.0 MB/s 
     |████████████████████████████████| 127 kB 73.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 125 kB 8.5 MB/s 
     |████████████████████████████████| 4.0 MB 9.2 MB/s 
     |████████████████████████████████| 6.6 MB 45.1 MB/s 
     |████████████████████████████████| 77 kB 8.1 MB/s 
 

#### pyrouge

In [ ]:
# install bheinzerling's pyrouge
!git clone https://github.com/bheinzerling/pyrouge
%cd pyrouge
!python setup.py install
# install missing dependency
!apt install libxml-parser-perl
%cd pyrouge
!git clone https://github.com/andersjo/pyrouge.git rouge

In [31]:
!pyrouge_set_rouge_path '/Users/imok/workspace/github/imOk/AI/pyrouge/pyrouge/rouge/tools/ROUGE-1.5.5'
%cd /Users/imok/workspace/github/imOk/AI/pyrouge/pyrouge/rouge/tools/ROUGE-1.5.5/data
!mv WordNet-2.0.exc.db WordNet-2.0.exc.db.orig
!perl WordNet-2.0-Exceptions/buildExeptionDB.pl ./WordNet-2.0-Exceptions ./smart_common_words.txt ./WordNet-2.0.exc.db

2022-04-15 12:29:28,173 [MainThread  ] [INFO ]  Set ROUGE home directory to /Users/imok/workspace/github/imOk/AI/pyrouge/pyrouge/rouge/tools/ROUGE-1.5.5.
/Users/imok/workspace/github/imOk/AI/pyrouge/pyrouge/rouge/tools/ROUGE-1.5.5/data


- colab에서 1분마다 자동 재연결
- 개발자 console에서 실행

```
function ClickConnect() {
    var buttons = document.querySelectorAll("colab-dialog.yes-no-dialog paper-button#cancel"); 
    buttons.forEach(function(btn) { 
        btn.click(); 
    }); 
    console.log("1분마다 자동 재연결"); 
    document.querySelector("colab-toolbar-button#connect").click(); 
} 
setInterval(ClickConnect,1000*60);
```

---

### (2) Train

In [32]:
!pwd

/Users/imok/workspace/github/imOk/AI/pyrouge/pyrouge/rouge/tools/ROUGE-1.5.5/data


In [33]:
cd /Users/imok/workspace/github/imOk/AI/pyrouge/pyrouge/rouge/tools/ROUGE-1.5.5

/Users/imok/workspace/github/imOk/AI/pyrouge/pyrouge/rouge/tools/ROUGE-1.5.5


In [34]:
ls

README.txt        ROUGE-1.5.5.pl*   data/
RELEASE-NOTE.txt  XML/              runROUGE-test.pl*


In [35]:
# PermissionError: [Errno 13] Permission denied: '/content/drive/MyDrive/AI/pyrouge/pyrouge/rouge/tools/ROUGE-1.5.5/ROUGE-1.5.5.pl'
!chmod 777 ROUGE-1.5.5.pl

In [36]:
cd /Users/imok/workspace/github/imOk/AI/KorBertSum/src

/Users/imok/workspace/github/imOk/AI/KorBertSum/src


In [ ]:
# m1 pytorch 에서 GPU 사용 불가
# AttributeError: module 'torch._C' has no attribute '_cuda_setDevice'
!python train.py -mode train -encoder classifier -dropout 0.1 \
-bert_data_path /Users/imok/workspace/github/imOk/AI/KorBertSum/bert_data/korean \
-model_path ../models/bert_classifier \
-lr 2e-3 -visible_gpus -1 -gpu_ranks -1 -world_size 1 -report_every 50 -save_checkpoint_steps 1000 \
-batch_size 1000 -decay_method noam -train_steps 1000 -accum_count 1 \
-log_file ../logs/bert_classifier -use_interval true -warmup_steps 8000 \
-bert_model /Users/imok/workspace/github/imOk/AI/model/001_bert_morp_pytorch \
-bert_config_path /Users/imok/workspace/github/imOk/AI/model/001_bert_morp_pytorch/bert_config.json -temp_dir .

### (3) Validation

In [ ]:
!python train.py -mode validate -encoder classifier -dropout 0.1 \
-bert_data_path /Users/imok/workspace/github/imOk/AI/KorBertSum/bert_data/korean \
-model_path ../models/bert_classifier \
-lr 2e-3 -visible_gpus 0 -gpu_ranks 0 -world_size 1 -report_every 50 -save_checkpoint_steps 1000 \
-batch_size 1000 -decay_method noam -train_steps 1000 -accum_count 1 \
-log_file ../logs/bert_classifier -use_interval true -warmup_steps 8000 \
-result_path ../results/korean \
-bert_model /Users/imok/workspace/github/imOk/AI/model/001_bert_morp_pytorch \
-bert_config_path /Users/imok/workspace/github/imOk/AI/model/001_bert_morp_pytorch/bert_config.json -temp_dir .

[2022-04-14 15:13:09,542 INFO] Loading checkpoint from ../models/bert_classifier/model_step_1000.pt
Namespace(accum_count=1, batch_size=1000, bert_config_path='/content/drive/MyDrive/AI/model/001_bert_morp_pytorch/bert_config.json', bert_data_path='/content/drive/MyDrive/AI/KorBertSum/bert_data/eunok/korean', bert_model='/content/drive/MyDrive/AI/model/001_bert_morp_pytorch', beta1=0.9, beta2=0.999, block_trigram=True, dataset='', decay_method='noam', dropout=0.1, encoder='classifier', ff_size=512, gpu_ranks=[0], heads=4, hidden_size=128, inter_layers=2, log_file='../logs/bert_classifier', lr=0.002, max_grad_norm=0, mode='validate', model_path='../models/bert_classifier', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=50, report_rouge=True, result_path='../results/korean', rnn_size=512, save_checkpoint_steps=1000, seed=666, temp_dir='.', test_all=False, test_from='', train_from='', train_steps=1000, use_interval=True, visible_gpus='0', warmup_steps=

### (4) Test

In [12]:
!python train.py -mode test -encoder classifier -dropout 0.1 \
-test_from ../models/bert_classifier/model_step_1000.pt \
-bert_data_path /Users/imok/workspace/github/imOk/AI/KorBertSum/bert_data/korean \
-model_path ../models/bert_classifier \
-lr 2e-3 -visible_gpus 0 -gpu_ranks 0 -world_size 1 -report_every 50 -save_checkpoint_steps 1000 \
-batch_size 1000 -decay_method noam -train_steps 1000 -accum_count 1 \
-log_file ../logs/bert_classifier -use_interval true -warmup_steps 8000 \
-result_path ../results/korean \
-bert_model /Users/imok/workspace/github/imOk/AI/model/001_bert_morp_pytorch \
-bert_config_path /Users/imok/workspace/github/imOk/AI/model/001_bert_morp_pytorch/bert_config.json -temp_dir .

[2022-04-14 16:28:20,111 INFO] Loading checkpoint from ../models/bert_classifier/model_step_1000.pt
Namespace(accum_count=1, batch_size=1000, bert_config_path='/content/drive/MyDrive/AI/model/001_bert_morp_pytorch/bert_config.json', bert_data_path='/content/drive/MyDrive/AI/KorBertSum/bert_data/eunok/korean', bert_model='/content/drive/MyDrive/AI/model/001_bert_morp_pytorch', beta1=0.9, beta2=0.999, block_trigram=True, dataset='', decay_method='noam', dropout=0.1, encoder='classifier', ff_size=512, gpu_ranks=[0], heads=4, hidden_size=128, inter_layers=2, log_file='../logs/bert_classifier', lr=0.002, max_grad_norm=0, mode='test', model_path='../models/bert_classifier', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=50, report_rouge=True, result_path='../results/korean', rnn_size=512, save_checkpoint_steps=1000, seed=666, temp_dir='.', test_all=False, test_from='../models/bert_classifier/model_step_1000.pt', train_from='', train_steps=1000, use_interv

#### 라벨 데이터 & 예측 데이터 확인


In [52]:
with open('/Users/imok/workspace/github/imOk/AI/KorBertSum/results/korean_step1000.gold','r') as f:
    gold = f.readlines()

In [53]:
with open('/Users/imok/workspace/github/imOk/AI/KorBertSum/results/korean_step1000.candidate','r') as f:
    candidate = f.readlines()

In [54]:
# 라벨 데이터
gold[5]

"'우리금융그룹 은 자회사 인 우리 은행 이 오 는 26 일 주식 시장 개장 전 시간 외 대량 매매 방식 으로 우리 금융 지분 4 %( 2889 만 707 주 ) 를 푸 본 생명 에 매각 한다고 25 일 밝혔 다 .'<q> '우리 은행 은 지난 10 일 자회사 였 던 우리 카드 를 우리 금융 지주 자회사 로 편입 시키 는 과정 에서 주당 1 만 2350 원 에 우리 금융 지주 지분 ( 5 . 8 %) 을 취득 했 다 . <q> '우리 금융 과 우리 은행 은 지난 4 월 부터 공동 태 스 크 포스 ( TF ) 를 만들 고 우리 은행 이 받 게 될 우리 금융 지주 주식 을 매각 하 는 방안 을 강구 해 왔 다 .\n"

In [55]:
# 예측 데이터
candidate[5]

"'우리금융그룹 은 자회사 인 우리 은행 이 오 는 26 일 주식 시장 개장 전 시간 외 대량 매매 방식 으로 우리 금융 지분 4 %( 2889 만 707 주 ) 를 푸 본 생명 에 매각 한다고 25 일 밝혔 다 .'<q>'우리 은행 은 지난 10 일 자회사 였 던 우리 카드 를 우리 금융 지주 자회사 로 편입 시키 는 과정 에서 주당 1 만 2350 원 에 우리 금융 지주 지분 ( 5 . 8 %) 을 취득 했 다 .<q>철저 한 사전 준비 와 적극 적 투자자 유치 활동 을 통해 글로벌 금융 시장 의 불안 에 도 불구 하 고 성공 적 지분 매각 이 라는 결실 을 맺 을 수 있 었 다고 우리 금융 측 은 설명 했 다 .'\n"

## 4. Test

In [56]:
import os

os.chdir('/Users/imok/workspace/github/imOk/AI/KorBertSum/src')

In [61]:
import torch
import numpy as np
from models import data_loader, model_builder
from models.model_builder import Summarizer
from others.logging import logger, init_logger
from models.data_loader import load_dataset
from transformers import BertConfig, BertTokenizer
from tensorboardX import SummaryWriter
from models.reporter import ReportMgr
from models.stats import Statistics
import easydict

In [62]:
!pwd

/Users/imok/workspace/github/imOk/AI/KorBertSum


In [63]:
cd /Users/imok/workspace/github/imOk/AI/KorBertSum

/Users/imok/workspace/github/imOk/AI/KorBertSum


In [101]:
def _tally_parameters(model):
    n_params = sum([p.nelement() for p in model.parameters()])
    return n_params

def build_trainer(args, device_id, model,
                  optim):
    """
    Simplify `Trainer` creation based on user `opt`s*
    Args:
        opt (:obj:`Namespace`): user options (usually from argument parsing)
        model (:obj:`onmt.models.NMTModel`): the model to train
        fields (dict): dict of fields
        optim (:obj:`onmt.utils.Optimizer`): optimizer used during training
        data_type (str): string describing the type of data
            e.g. "text", "img", "audio"
        model_saver(:obj:`onmt.models.ModelSaverBase`): the utility object
            used to save the model
    """
    device = "cpu" if args.visible_gpus == '-1' else "cuda"


    grad_accum_count = args.accum_count
    n_gpu = args.world_size

    if device_id >= 0:
        gpu_rank = int(args.gpu_ranks[device_id])
    else:
        gpu_rank = 0
        n_gpu = 0

    print('gpu_rank %d' % gpu_rank)

    tensorboard_log_dir = args.model_path

    writer = SummaryWriter(tensorboard_log_dir, comment="Unmt")

    report_manager = ReportMgr(args.report_every, start_time=-1, tensorboard_writer=writer)

    trainer = Trainer(args, model, optim, grad_accum_count, n_gpu, gpu_rank, report_manager)

    # print(tr)
    if (model):
        n_params = _tally_parameters(model)
        logger.info('* number of parameters: %d' % n_params)

    return trainer


class Trainer(object):
    """
    Class that controls the training process.

    Args:
            model(:py:class:`onmt.models.model.NMTModel`): translation model
                to train
            train_loss(:obj:`onmt.utils.loss.LossComputeBase`):
               training loss computation
            valid_loss(:obj:`onmt.utils.loss.LossComputeBase`):
               training loss computation
            optim(:obj:`onmt.utils.optimizers.Optimizer`):
               the optimizer responsible for update
            trunc_size(int): length of truncated back propagation through time
            shard_size(int): compute loss in shards of this size for efficiency
            data_type(string): type of the source input: [text|img|audio]
            norm_method(string): normalization methods: [sents|tokens]
            grad_accum_count(int): accumulate gradients this many times.
            report_manager(:obj:`onmt.utils.ReportMgrBase`):
                the object that creates reports, or None
            model_saver(:obj:`onmt.models.ModelSaverBase`): the saver is
                used to save a checkpoint.
                Thus nothing will be saved if this parameter is None
    """

    def __init__(self,  args, model,  optim,
                  grad_accum_count=1, n_gpu=1, gpu_rank=1,
                  report_manager=None):
        # Basic attributes.
        self.args = args
        self.save_checkpoint_steps = args.save_checkpoint_steps
        self.model = model
        self.optim = optim
        self.grad_accum_count = grad_accum_count
        self.n_gpu = n_gpu
        self.gpu_rank = gpu_rank
        self.report_manager = report_manager

        self.loss = torch.nn.BCELoss(reduction='none')
        assert grad_accum_count > 0
        # Set model in training mode.
        if (model):
            self.model.train()

    def summ(self, test_iter, step, cal_lead=False, cal_oracle=False):
        """ Validate model.
            valid_iter: validate data iterator
        Returns:
            :obj:`nmt.Statistics`: validation loss statistics
        """
      # Set model in validating mode.
        def _get_ngrams(n, text):
            ngram_set = set()
            text_length = len(text)
            max_index_ngram_start = text_length - n
            for i in range(max_index_ngram_start + 1):
                ngram_set.add(tuple(text[i:i + n]))
            return ngram_set

        def _block_tri(c, p):
            tri_c = _get_ngrams(3, c.split())
            for s in p:
                tri_s = _get_ngrams(3, s.split())
                if len(tri_c.intersection(tri_s))>0:
                    return True
            return False

        if (not cal_lead and not cal_oracle):
            self.model.eval()
        stats = Statistics()

        with torch.no_grad():
            for batch in test_iter:
                src = batch.src
                labels = batch.labels
                segs = batch.segs
                clss = batch.clss
                mask = batch.mask
                mask_cls = batch.mask_cls

                if (cal_lead):
                    selected_ids = [list(range(batch.clss.size(1)))] * batch.batch_size
                elif (cal_oracle):
                    selected_ids = [[j for j in range(batch.clss.size(1)) if labels[i][j] == 1] for i in
                                    range(batch.batch_size)]
                else:
                    sent_scores, mask = self.model(src, segs, clss, mask, mask_cls)
                    sent_scores = sent_scores + mask.float()
                    sent_scores = sent_scores.cpu().data.numpy()
                    selected_ids = np.argsort(-sent_scores, 1)
        return selected_ids



    def _gradient_accumulation(self, true_batchs, normalization, total_stats,
                               report_stats):
        if self.grad_accum_count > 1:
            self.model.zero_grad()

        for batch in true_batchs:
            if self.grad_accum_count == 1:
                self.model.zero_grad()

            src = batch.src
            labels = batch.labels
            segs = batch.segs
            clss = batch.clss
            mask = batch.mask
            mask_cls = batch.mask_cls

            sent_scores, mask = self.model(src, segs, clss, mask, mask_cls)

            loss = self.loss(sent_scores, labels.float())
            loss = (loss*mask.float()).sum()
            (loss/loss.numel()).backward()
            # loss.div(float(normalization)).backward()

            batch_stats = Statistics(float(loss.cpu().data.numpy()), normalization)


            total_stats.update(batch_stats)
            report_stats.update(batch_stats)

            # 4. Update the parameters and statistics.
            if self.grad_accum_count == 1:
                # Multi GPU gradient gather
                if self.n_gpu > 1:
                    grads = [p.grad.data for p in self.model.parameters()
                             if p.requires_grad
                             and p.grad is not None]
                    distributed.all_reduce_and_rescale_tensors(
                        grads, float(1))
                self.optim.step()

        # in case of multi step gradient accumulation,
        # update only after accum batches
        if self.grad_accum_count > 1:
            if self.n_gpu > 1:
                grads = [p.grad.data for p in self.model.parameters()
                         if p.requires_grad
                         and p.grad is not None]
                distributed.all_reduce_and_rescale_tensors(
                    grads, float(1))
            self.optim.step()

    def _save(self, step):
        real_model = self.model
        # real_generator = (self.generator.module
        #                   if isinstance(self.generator, torch.nn.DataParallel)
        #                   else self.generator)

        model_state_dict = real_model.state_dict()
        # generator_state_dict = real_generator.state_dict()
        checkpoint = {
            'model': model_state_dict,
            # 'generator': generator_state_dict,
            'opt': self.args,
            'optim': self.optim,
        }
        checkpoint_path = os.path.join(self.args.model_path, 'model_step_%d.pt' % step)
        logger.info("Saving checkpoint %s" % checkpoint_path)
        # checkpoint_path = '%s_step_%d.pt' % (FLAGS.model_path, step)
        if (not os.path.exists(checkpoint_path)):
            torch.save(checkpoint, checkpoint_path)
            return checkpoint, checkpoint_path

    def _start_report_manager(self, start_time=None):
        """
        Simple function to start report manager (if any)
        """
        if self.report_manager is not None:
            if start_time is None:
                self.report_manager.start()
            else:
                self.report_manager.start_time = start_time

    def _maybe_gather_stats(self, stat):
        """
        Gather statistics in multi-processes cases

        Args:
            stat(:obj:onmt.utils.Statistics): a Statistics object to gather
                or None (it returns None in this case)

        Returns:
            stat: the updated (or unchanged) stat object
        """
        if stat is not None and self.n_gpu > 1:
            return Statistics.all_gather_stats(stat)
        return stat

    def _maybe_report_training(self, step, num_steps, learning_rate,
                               report_stats):
        """
        Simple function to report training stats (if report_manager is set)
        see `onmt.utils.ReportManagerBase.report_training` for doc
        """
        if self.report_manager is not None:
            return self.report_manager.report_training(
                step, num_steps, learning_rate, report_stats,
                multigpu=self.n_gpu > 1)

    def _report_step(self, learning_rate, step, train_stats=None,
                     valid_stats=None):
        """
        Simple function to report stats (if report_manager is set)
        see `onmt.utils.ReportManagerBase.report_step` for doc
        """
        if self.report_manager is not None:
            return self.report_manager.report_step(
                learning_rate, step, train_stats=train_stats,
                valid_stats=valid_stats)

    def _maybe_save(self, step):
        """
        Save the model if a model saver is set
        """
        if self.model_saver is not None:
            self.model_saver.maybe_save(step)

class BertData():
    def __init__(self):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
        self.sep_vid = self.tokenizer.vocab['[SEP]']
        self.cls_vid = self.tokenizer.vocab['[CLS]']
        self.pad_vid = self.tokenizer.vocab['[PAD]']

    def preprocess(self, src):

        if (len(src) == 0):
            return None

        original_src_txt = [' '.join(s) for s in src]
        idxs = [i for i, s in enumerate(src) if (len(s) > 1)]

        src = [src[i][:2000] for i in idxs]
        src = src[:1000]

        if (len(src) < 3):
            return None

        src_txt = [' '.join(sent) for sent in src]
        text = ' [SEP] [CLS] '.join(src_txt)
        src_subtokens = self.tokenizer.tokenize(text)
        src_subtokens = src_subtokens[:510]
        src_subtokens = ['[CLS]'] + src_subtokens + ['[SEP]']

        src_subtoken_idxs = self.tokenizer.convert_tokens_to_ids(src_subtokens)
        _segs = [-1] + [i for i, t in enumerate(src_subtoken_idxs) if t == self.sep_vid]
        segs = [_segs[i] - _segs[i - 1] for i in range(1, len(_segs))]
        segments_ids = []
        for i, s in enumerate(segs):
            if (i % 2 == 0):
                segments_ids += s * [0]
            else:
                segments_ids += s * [1]
        cls_ids = [i for i, t in enumerate(src_subtoken_idxs) if t == self.cls_vid]
        labels = None
        src_txt = [original_src_txt[i] for i in idxs]
        tgt_txt = None
        return src_subtoken_idxs, labels, segments_ids, cls_ids, src_txt, tgt_txt

def _lazy_dataset_loader(pt_file):
    yield  pt_file

### Params

In [102]:
args = easydict.EasyDict({
    "encoder":'classifier',
    "mode":'test',
    "bert_data_path":'/Users/imok/workspace/github/imOk/AI/KorBertSum/bert_data/korean',
    "model_path":'../models/bert_classifier',
    "result_path":'./results',
    "temp_dir":'./temp',
    "batch_size":1000,
    "use_interval":True,
    "hidden_size":128,
    "ff_size":512,
    "heads":4,
    "inter_layers":2,
    "rnn_size":512,
    "param_init":0,
    "param_init_glorot":True,
    "dropout":0.1,
    "optim":'adam',
    "lr":2e-3,
    "report_every":1,
    "save_checkpoint_steps":5,
    "block_trigram":True,
    "recall_eval":False,
    "bert_model":'/Users/imok/workspace/github/imOk/AI/model/001_bert_morp_pytorch',
    "bert_config_path": '/Users/imok/workspace/github/imOk/AI/model/001_bert_morp_pytorch/bert_config.json',
    "accum_count":1,
    "world_size":1,
    "visible_gpus":'-1',
    "gpu_ranks":'0',
    "log_file":'/Users/imok/workspace/github/imOk/AI/KorBertSum/logs/log.log',
    "test_from":'/Users/imok/workspace/github/imOk/AI/KorBertSum/models/bert_classifier/model_step_1000.pt'
})
model_flags = ['hidden_size', 'ff_size', 'heads', 'inter_layers','encoder','ff_actv', 'use_interval','rnn_size']

### Test code

In [103]:
def test(args, input_list, device_id, pt, step):
    init_logger(args.log_file)
    device = "cpu" if args.visible_gpus == '-1' else "cuda"
    device_id = 0 if device == "cuda" else -1

    cp = args.test_from
    try:
        step = int(cp.split('.')[-2].split('_')[-1])
    except:
        step = 0

    device = "cpu" if args.visible_gpus == '-1' else "cuda"
    if (pt != ''):
        test_from = pt
    else:
        test_from = args.test_from
    logger.info('Loading checkpoint from %s' % test_from)
    checkpoint = torch.load(test_from, map_location=lambda storage, loc: storage)
    opt = vars(checkpoint['opt'])
    for k in opt.keys():
        if (k in model_flags):
            setattr(args, k, opt[k])

    config = BertConfig.from_pretrained('bert-base-multilingual-cased')
    model = Summarizer(args, device, load_pretrained_bert=False, bert_config = config)
    model.load_cp(checkpoint)
    model.eval()

    test_iter = data_loader.Dataloader(args, _lazy_dataset_loader(input_list),
                                args.batch_size, device,
                                shuffle=False, is_test=True)
    trainer = build_trainer(args, device_id, model, None)
    result = trainer.summ(test_iter,step)
    return result, input_list

args.gpu_ranks = [int(i) for i in args.gpu_ranks.split(',')]
os.environ["CUDA_VISIBLE_DEVICES"] = args.visible_gpus

In [104]:
def txt2input(text):
    data = list(filter(None, text.split('\n')))
    bertdata = BertData()
    txt_data = bertdata.preprocess(data)
    data_dict = {"src":txt_data[0],
               "labels":[0,1,2],
               "segs":txt_data[2],
               "clss":txt_data[3],
               "src_txt":txt_data[4],
               "tgt_txt":None}
    input_data = []
    input_data.append(data_dict)
    return input_data

In [107]:
text = '''
여덟 살짜리 소녀가 러시아의 침공으로 피해를 입은 우크라이나 어린이들을 위해 두 팔을 걷어붙이고 나섰다.
6일 영국 bbc에 따르면 링컨셔주에 는 아바 로즈 클라크는 우크라이나 친구를 돕기 위해 4시간 챌린지를 시작했다.
이 도전은 요크셔주와 링컨셔주를 잇는 전체 길이 20마일 규모의 험버 대교를 자전거로 건너는 것.
도전 첫날이었던 지난 5일 아바 로즈는 험버 대교의 6마일을 횡단하는 데 성공했다.
그는 우크라이나 어린이들이 평소 가지고 놀던 장난감과 물건을 버리고 피란하는 모습을 보며 도움을 줄 방법을 고민하던 중 챌린지를 생각해냈다고 말했다.
유니세프를 통해 1000파운드를 목표로 시작한 아바 로즈의 모금 챌린지 누적 금액은 7일 기준 1065파운드를 넘겼다.
험버 대교를 건너는 아바 로즈의 모습을 상상해서 그려주세요.
'''

In [108]:
input_data = txt2input(text)
sum_list = test(args, input_data, -1, '', None)
sum_list[0]

[2022-04-15 13:44:25,365 INFO] Loading checkpoint from /Users/imok/workspace/github/imOk/AI/KorBertSum/models/bert_classifier/model_step_1000.pt
[2022-04-15 13:44:27,074 INFO] loading archive file /Users/imok/workspace/github/imOk/AI/model/001_bert_morp_pytorch
[2022-04-15 13:44:27,077 INFO] Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 30349
}

[2022-04-15 13:44:29,761 INFO] * number of parameters: 109350145


gpu_rank 0


array([[6, 3, 5, 0, 1, 4, 2]])

### Result

In [109]:
[list(filter(None, text.split('\n')))[i] for i in sum_list[0][0][:3]]

['험버 대교를 건너는 아바 로즈의 모습을 상상해서 그려주세요.',
 '도전 첫날이었던 지난 5일 아바 로즈는 험버 대교의 6마일을 횡단하는 데 성공했다.',
 '유니세프를 통해 1000파운드를 목표로 시작한 아바 로즈의 모금 챌린지 누적 금액은 7일 기준 1065파운드를 넘겼다.']